In [1]:
import requests
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import csv
import time
import json
from pandas import json_normalize

In [18]:
data = pd.read_csv("임대주택단지.csv", encoding='cp949')
display(data)

,Unnamed: 0,전용면적,월임대료,총세대수,데이터순번,단지명,최초입주년월,지역명,세대수,전체건수,공급유형명,임대보증금
0,0,46.7100,358790,873,1,서울강남 3블록,201311.0,서울특별시 강남구,72,19,국민임대,43231000
1,1,46.0500,358790,873,2,서울강남 3블록,201311.0,서울특별시 강남구,64,19,국민임대,43231000
2,2,46.0100,358790,873,3,서울강남 3블록,201311.0,서울특별시 강남구,180,19,국민임대,43231000
3,3,36.6200,252360,873,4,서울강남 3블록,201311.0,서울특별시 강남구,84,19,국민임대,20188000
4,4,36.0800,252360,873,5,서울강남 3블록,201311.0,서울특별시 강남구,448,19,국민임대,20188000
...,...,...,...,...,...,...,...,...,...,...,...,...
2981,2981,46.7203,81540,151,66,함덕진우,NaN,제주특별자치도 제주시,16,70,국민임대,14833000
2982,2982,37.2671,50350,151,67,함덕진우,NaN,제주특별자치도 제주시,3,70,국민임대,9221000
2983,2983,37.2638,50350,151,68,함덕진우,NaN,제주특별자치도 제주시,3,70,국민임대,9221000
2984,2984,37.2011,55150,151,69,함덕진우,NaN,제주특별자치도 제주시,4,70,국민임대,10036000


In [19]:
data.shape, data.drop_duplicates().shape # 중복 체크

((2986, 12), (2986, 12))

In [27]:
data = data.rename(columns={'지역명':'지역'})
display(data)

df서울 = data[data['지역'].str.contains('서울특별시')]
df서울['지역'] = '서울특별시'

df부산 = data[data['지역'].str.contains('부산광역시')]
df부산['지역'] = '부산광역시'

df대구 = data[data['지역'].str.contains('대구광역시')]
df대구['지역'] = '대구광역시'

df인천 = data[data['지역'].str.contains('인천광역시')]
df인천['지역'] = '인천광역시'

df광주 = data[data['지역'].str.contains('광주광역시')]
df광주['지역'] = '광주광역시'

df대전 = data[data['지역'].str.contains('대전광역시')]
df대전['지역'] = '대전광역시'

df울산 = data[data['지역'].str.contains('울산광역시')]
df울산['지역'] = '울산광역시'

df세종 = data[data['지역'].str.contains('세종시')]
df세종['지역'] = '세종특별자치시'

df경기 = data[data['지역'].str.contains('경기도')]
df경기['지역'] = '경기도'

df강원 = data[data['지역'].str.contains('강원도')]
df강원['지역'] = '강원도'

df충북 = data[data['지역'].str.contains('충청북도')]
df충북['지역'] = '충청북도'

df충남 = data[data['지역'].str.contains('충청남도')]
df충남['지역'] = '충청남도'

df전북 = data[data['지역'].str.contains('전라북도')]
df전북['지역'] = '전라북도'

df전남 = data[data['지역'].str.contains('전라남도')]
df전남['지역'] = '전라남도'

df경북 = data[data['지역'].str.contains('경상북도')]
df경북['지역'] = '경상북도'

df경남 = data[data['지역'].str.contains('경상남도')]
df경남['지역'] = '경상남도'

df제주 = data[data['지역'].str.contains('제주특별자치도')]
df제주['지역'] = '제주특별자치도'

result = pd.concat([df서울, df부산, df대구, df인천, df광주, df대전, df울산, 
                    df세종, df경기, df강원, df충북, df충남, df전북, df전남, df경북, df경남, df제주],ignore_index=True)
result = result.drop(['Unnamed: 0'], axis=1)
result

,Unnamed: 0,전용면적,월임대료,총세대수,데이터순번,단지명,최초입주년월,지역,세대수,전체건수,공급유형명,임대보증금
0,0,46.7100,358790,873,1,서울강남 3블록,201311.0,서울특별시 강남구,72,19,국민임대,43231000
1,1,46.0500,358790,873,2,서울강남 3블록,201311.0,서울특별시 강남구,64,19,국민임대,43231000
2,2,46.0100,358790,873,3,서울강남 3블록,201311.0,서울특별시 강남구,180,19,국민임대,43231000
3,3,36.6200,252360,873,4,서울강남 3블록,201311.0,서울특별시 강남구,84,19,국민임대,20188000
4,4,36.0800,252360,873,5,서울강남 3블록,201311.0,서울특별시 강남구,448,19,국민임대,20188000
...,...,...,...,...,...,...,...,...,...,...,...,...
2981,2981,46.7203,81540,151,66,함덕진우,NaN,제주특별자치도 제주시,16,70,국민임대,14833000
2982,2982,37.2671,50350,151,67,함덕진우,NaN,제주특별자치도 제주시,3,70,국민임대,9221000
2983,2983,37.2638,50350,151,68,함덕진우,NaN,제주특별자치도 제주시,3,70,국민임대,9221000
2984,2984,37.2011,55150,151,69,함덕진우,NaN,제주특별자치도 제주시,4,70,국민임대,10036000


<ipython-input-27-aac23fd0aa8f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df서울['지역'] = '서울특별시'
<ipython-input-27-aac23fd0aa8f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df부산['지역'] = '부산광역시'
<ipython-input-27-aac23fd0aa8f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

,전용면적,월임대료,총세대수,데이터순번,단지명,최초입주년월,지역,세대수,전체건수,공급유형명,임대보증금
0,46.7100,358790,873,1,서울강남 3블록,201311.0,서울특별시,72,19,국민임대,43231000
1,46.0500,358790,873,2,서울강남 3블록,201311.0,서울특별시,64,19,국민임대,43231000
2,46.0100,358790,873,3,서울강남 3블록,201311.0,서울특별시,180,19,국민임대,43231000
3,36.6200,252360,873,4,서울강남 3블록,201311.0,서울특별시,84,19,국민임대,20188000
4,36.0800,252360,873,5,서울강남 3블록,201311.0,서울특별시,448,19,국민임대,20188000
...,...,...,...,...,...,...,...,...,...,...,...
2981,46.7203,81540,151,66,함덕진우,NaN,제주특별자치도,16,70,국민임대,14833000
2982,37.2671,50350,151,67,함덕진우,NaN,제주특별자치도,3,70,국민임대,9221000
2983,37.2638,50350,151,68,함덕진우,NaN,제주특별자치도,3,70,국민임대,9221000
2984,37.2011,55150,151,69,함덕진우,NaN,제주특별자치도,4,70,국민임대,10036000


In [28]:
result.to_csv('임대주택단지_전처리.csv', encoding='cp949')